In [21]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [22]:
import pandas as pd

In [23]:
df = pd.read_csv(r'artifacts/customer_churn_removed_col.csv')

In [24]:
df.dtypes

Age                             int64
Gender                         object
Location                       object
Subscription_Length_Months      int64
Monthly_Bill                  float64
Total_Usage_GB                  int64
Churn                           int64
dtype: object

## Entity

In [25]:
from pathlib import Path
from dataclasses import dataclass

In [26]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    data_dir:Path
    schema_check:dict
    make_data:dict

## configuration manager

In [27]:
from churnPredictor.constants import *
from churnPredictor.utils import *

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):


        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.INDEPENDENT_FEATURES

        create_dirs([config.root_dir])
        

        data_validation_config =  DataValidationConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir,
            schema_check = schema,
            make_data=config.make_data
        )

        return data_validation_config
        
        
        

## components

In [29]:
class DataValidataion:
    def __init__(self,config_in :DataValidationConfig):
        self.config_in = config_in
    
    def validate_features(self) -> bool:
        try:
            validation_status = False
            df = pd.read_csv(self.config_in.data_dir)
            all_features = list(df.columns)
            all_schema = self.config_in.schema_check

            for cols in all_features:
                if cols not in all_schema:
                    validation_status = False
                    logger.info(f'Validation Status is {validation_status}')
                
                else:
                    validation_status = True
                    logger.info(f'Validation Status is {validation_status}')

            return validation_status
        except Exception as e:
            raise CustomException(e)
        

## pipeline

In [30]:
try:
    config = ConfigurationManager()
    data_val_confg = config.get_data_validation_config()
    data_validation = DataValidataion(config_in=data_val_confg)
    data_validation.validate_features()

except Exception as e:
    raise CustomException(e)

[2023-09-15 14:42:43,217: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-09-15 14:42:43,220: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-09-15 14:42:43,222: INFO: utils: yaml file: schema.yaml loaded successfully]
[2023-09-15 14:42:43,224: INFO: utils: Created artifacts]


CustomException: Error occured in python script name [C:\Users\Gyanprakash\AppData\Local\Temp\ipykernel_15152\320748816.py] line number [3] error message ["'ConfigBox' object has no attribute 'independent_features'"]